In [ ]:

from src.catalogs import (
    JapanSlowSlipCatalog,
    MexicoSlowSlipCatalog, 
    XieSlowSlipCatalog,
    WilliamsSlowSlipCatalog,
    MichelSlowSlipCatalog,   # too few earthquakes to estaplish rate changes 
    OkadaAlaskaSlowSlipCatalog,
)

from src.data import (
    AllSlabs, 
    Catalog,
    EarthquakeCatalog,
    Scaling,
)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import string
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import gaussian_kde, kstest

from typing import Union

import src
from pathlib import Path
base_dir = Path(src.__file__).parents[1]

dpi = 100
mpl.rc('figure',dpi=dpi)
plt.style.use('tableau-colorblind10')
mpl.rc('font',size=9)
figure_save_dir = base_dir/'Docs'/'Figures_SCEC'
save_figures = True
def savefig(fig_name, save_fig_bool = save_figures):
    plt.savefig(figure_save_dir/f'{fig_name}.pdf', transparent=True,bbox_inches='tight')

In [ ]:
all_slabs = AllSlabs()

minimum_magnitude = 4.0

metadata = {
    "starttime": '1994-01-01',
    "endtime": '2023-01-01',
    "latitude_range": [-90,90],
    "longitude_range": [-180,180],
    "minimum_magnitude":minimum_magnitude 
}

In [ ]:
earthquakes = EarthquakeCatalog(
    filename=base_dir / "Datasets" / "Seismicity_datasets" / "global_earthquakes.csv",
    kwargs=metadata,
)

DEPTH_SUBSET = 'all'
assert DEPTH_SUBSET in ['deep','shallow','all']

DURATION_SUBSET = 'all'
assert DURATION_SUBSET in ['short term', 'long term', 'all']

SLOWSLIP_MAGNITUDE_CUTOFF = 5.0
assert type(SLOWSLIP_MAGNITUDE_CUTOFF) is float

# Duplicate filtering
DUPLICATE_RADIUS = 70
DUPLICATE_TIME = 40

# Spatio-temporal emporal analysis:
TIME_WINDOW = 365 # days
SMOOTHING_BANDWIDTH_DAYS = 5 # days

SPACE_WINDOW_BIG = 2000 # km for spatial stack
DISTANCE_TO_SLAB = 20 # km 

IMPUTE_DURATION = True
REPRESENTATIVE_SSE_SIZE = 100 # km for temporal stack

BACKGROUND_DURATION = 365

MIN_NUMBER_OF_EVENTS_FOR_RATE_CHANGE = 4